# Building irregular bonds

(Based on [a question](http://quant.stackexchange.com/questions/11090/) by _Stack Exchange_ user user7922, [another](http://quant.stackexchange.com/questions/9080/) by user Lisa Ann, and [yet another](https://sourceforge.net/p/quantlib/mailman/message/36170786/) asked by Anthony Calleja on the QuantLib mailing list.  Thanks!)

In [1]:
from QuantLib import *
from datetime import date
from pandas import DataFrame
import utils

Let me just define a small helper function before starting.  It's just for visualization, nothing to write about.

In [2]:
def rate_if_available(c):
    c = as_coupon(c)
    return utils.format_rate(c.rate()) if c else ''

#### The first question

user7922 had to price a bond that, curiously, had a last coupon date before the maturity date; e.g., the last coupon date is April 20th, 2020 and maturity date is April 20th, 2021. Yes, it's strange, but who are we to judge?

There's no way to express this directly in QuantLib, but we can get it with some work. In case of a fixed-rate bond, and if we want to do the simplest thing that can possibly work, we can cancel the last coupon by giving it a rate of 0%.

In [3]:
today = Date(8, October, 2014)
Settings.instance().evaluationDate = today

In [4]:
issueDate = today+7
maturityDate = issueDate+Period(10,Years)

schedule = Schedule(issueDate, maturityDate,
                    Period(Annual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

The schedule we just built gives us the correct maturity date, as well as the date where we want the last real coupon.
Now for the bond:

In [5]:
settlementDays = 3
faceAmount = 100
paymentDayCounter = Thirty360()

coupon_rate = 0.02
N = len(schedule)-1   # number of coupons
coupons = [coupon_rate]*(N-1) + [0.0]

bond = FixedRateBond(settlementDays,
                     faceAmount,
                     schedule,
                     coupons,
                     paymentDayCounter)

In [6]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
            for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",2.00 %,2.000000
,"October 17th, 2016",2.00 %,2.011111
,"October 16th, 2017",2.00 %,1.994444
,"October 15th, 2018",2.00 %,1.994444
,"October 15th, 2019",2.00 %,2.000000
,"October 15th, 2020",2.00 %,2.000000
,"October 15th, 2021",2.00 %,2.000000
,"October 17th, 2022",2.00 %,2.011111
,"October 16th, 2023",2.00 %,1.994444
,"October 15th, 2024",0.00 %,0.000000


The same trick also works for floating-rate coupons, if we use a null gearing for the last coupon:

In [7]:
euribor_curve = FlatForward(0, TARGET(), 0.002, Actual360())
index = Euribor1Y(YieldTermStructureHandle(euribor_curve))
N = len(schedule)-1   # number of coupons
gearings = [1.0]*(N-1) + [0.0]
bond = FloatingRateBond(settlementDays = 3,
                        faceAmount = 100,
                        schedule = schedule,
                        index = index,
                        paymentDayCounter = Thirty360(),
                        paymentConvention = Following,
                        fixingDays = index.fixingDays(),
                        gearings = gearings,
                        spreads = [],
                        caps= [],
                        floors = [],
                        inArrears = False,
                        redemption = 100.0,
                        issueDate = issueDate)

In [8]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
           for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",0.20 %,0.200203
,"October 17th, 2016",0.20 %,0.201317
,"October 16th, 2017",0.20 %,0.199646
,"October 15th, 2018",0.20 %,0.199646
,"October 15th, 2019",0.20 %,0.200203
,"October 15th, 2020",0.20 %,0.200203
,"October 15th, 2021",0.20 %,0.200203
,"October 17th, 2022",0.20 %,0.201316
,"October 16th, 2023",0.20 %,0.199646
,"October 15th, 2024",0.00 %,0.000000


However, that's a bit sloppy.  The coupon paying a null rate looks strange, and might confuse cash-flow analysis.  It's better, even if it takes a bit more work, to remove the coupon altogether.  We can get the cash flows from the bond we created...

In [9]:
cashflows = list(bond.cashflows())

...delete the ones we don't want to keep, that is, the one before the last (the last being the redemption)...

In [10]:
del cashflows[-2]

...and use the cleaned-up cash flows to create a new bond:

In [11]:
bond = Bond(3, TARGET(), 100.0,
            maturityDate, issueDate,
            cashflows)

This gives us the coupons we wanted:

In [12]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
            for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",0.20 %,0.200203
,"October 17th, 2016",0.20 %,0.201317
,"October 16th, 2017",0.20 %,0.199646
,"October 15th, 2018",0.20 %,0.199646
,"October 15th, 2019",0.20 %,0.200203
,"October 15th, 2020",0.20 %,0.200203
,"October 15th, 2021",0.20 %,0.200203
,"October 17th, 2022",0.20 %,0.201316
,"October 16th, 2023",0.20 %,0.199646
,"October 15th, 2024",,100.000000


#### The second question

Lisa Ann had to price a more common instrument (namely, a fixed-to-floater) for which, however, there's no specific class in the library. In this case, too, we can create the required bond by manipulating coupons; and in this case, too, we can choose how much work to do.

Let's say the bond pays three fixed-rate coupon first and then seven floating-rate coupons. We might create the fixed-rate coupons...

In [13]:
schedule = Schedule(issueDate, issueDate+Period(3,Years),
                    Period(Annual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

fixed = FixedRateLeg(schedule = schedule,
                     dayCount = Actual360(),
                     nominals = [100.0],
                     couponRates = [0.02])

...then the floating-rate coupons...

In [14]:
schedule = Schedule(issueDate+Period(3,Years), maturityDate,
                    Period(Annual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

floating = IborLeg(nominals = [100.0],
                   schedule = schedule,
                   index = index,
                   spreads = [0.001])

...and finally put them together, add the redemption, and build a custom bond:

In [15]:
bond = Bond(3, TARGET(), 100.0,
            maturityDate, issueDate,
            fixed + floating + (Redemption(100.0, maturityDate),))

In [16]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
            for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",2.00 %,2.027778
,"October 17th, 2016",2.00 %,2.044444
,"October 16th, 2017",2.00 %,2.022222
,"October 15th, 2018",0.30 %,0.303538
,"October 15th, 2019",0.30 %,0.304372
,"October 15th, 2020",0.30 %,0.305207
,"October 15th, 2021",0.30 %,0.304372
,"October 17th, 2022",0.30 %,0.306041
,"October 16th, 2023",0.30 %,0.303538
,"October 15th, 2024",0.30 %,0.304372


However, I'm not very comfortable with building the coupons with two half-schedules; I haven't looked very hard for a counter-example, but I suspect that some combination of holidays and business-day conventions might cause the coupon dates to be off.

A safer alternative would be to build both fixed and floating coupons over the full bond schedule, and just keep those we need:

In [17]:
schedule = Schedule(issueDate, maturityDate,
                    Period(Annual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

fixed = FixedRateLeg(schedule = schedule,
                     dayCount = Actual360(),
                     nominals = [100.0],
                     couponRates = [0.02])

floating = IborLeg(nominals = [100.0],
                   schedule = schedule,
                   index = index,
                   spreads = [0.001])

cashflows = fixed[:3] + floating[3:] + (Redemption(100.0, maturityDate),)

In [18]:
bond = Bond(3, TARGET(), 100.0,
            maturityDate, issueDate, cashflows)

In [19]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
            for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",2.00 %,2.027778
,"October 17th, 2016",2.00 %,2.044444
,"October 16th, 2017",2.00 %,2.022222
,"October 15th, 2018",0.30 %,0.303538
,"October 15th, 2019",0.30 %,0.304372
,"October 15th, 2020",0.30 %,0.305207
,"October 15th, 2021",0.30 %,0.304372
,"October 17th, 2022",0.30 %,0.306041
,"October 16th, 2023",0.30 %,0.303538
,"October 15th, 2024",0.30 %,0.304372


Also, in a pinch (for instance, if you're using the QuantLib Excel module and can't create custom bonds easily) a fixed-rate coupon can be approximated by a floating-rate coupon with a null gearing and a spread equal to the desired rate, so you might get the same result this way:

In [20]:
schedule = Schedule(issueDate, maturityDate,
                    Period(Annual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

gearings = [0.0]*3 + [1.0]*7
spreads = [0.02]*3 + [0.001]*7

bond = FloatingRateBond(settlementDays = 3,
                        faceAmount = 100,
                        schedule = schedule,
                        index = index,
                        paymentDayCounter = Actual360(),
                        paymentConvention = Following,
                        fixingDays = index.fixingDays(),
                        gearings = gearings,
                        spreads = spreads,
                        caps= [],
                        floors = [],
                        inArrears = False,
                        redemption = 100.0,
                        issueDate = issueDate)

In [21]:
DataFrame([ (c.date(), rate_if_available(c), c.amount())
           for c in bond.cashflows() ],
          columns = ('date', 'rate', 'amount'),
          index=['']*len(bond.cashflows()))

,date,rate,amount
,"October 15th, 2015",2.00 %,2.027778
,"October 17th, 2016",2.00 %,2.044444
,"October 16th, 2017",2.00 %,2.022222
,"October 15th, 2018",0.30 %,0.303538
,"October 15th, 2019",0.30 %,0.304372
,"October 15th, 2020",0.30 %,0.305207
,"October 15th, 2021",0.30 %,0.304372
,"October 17th, 2022",0.30 %,0.306041
,"October 16th, 2023",0.30 %,0.303538
,"October 15th, 2024",0.30 %,0.304372


However, I don't suggest doing this if you can get actual fixed- and floating-rate coupons.

#### The third question

This one requires a bit more work (and involved a swap, instead of a bond, but it doesn't matter; you can build custom swaps with the `Swap` class).  Anthony needed a floating leg paying 6-months Euribor, but with a short initial stub paying the fixing of 3-months Euribor instead.  In a vanilla leg, even with the correct schedule, the first coupon would pay the 6-months fixing instead: 

In [22]:
euribor_curve_3m = FlatForward(0, TARGET(), 0.0015, Actual360())
index_3m = Euribor3M(YieldTermStructureHandle(euribor_curve_3m))

euribor_curve_6m = FlatForward(0, TARGET(), 0.0020, Actual360())
index_6m = Euribor6M(YieldTermStructureHandle(euribor_curve_6m))

In [23]:
startDate = today + 7
endDate = startDate + Period(3,Months) + Period(5,Years)
schedule = Schedule(startDate, endDate,
                    Period(Semiannual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

cashflows = IborLeg(nominals = [100.0],
                    schedule = schedule,
                    index = index_6m)

In [24]:
DataFrame([ (c.date(),
             as_coupon(c).accrualStartDate(), as_coupon(c).accrualEndDate(),
             utils.format_rate(as_coupon(c).rate()), c.amount())
            for c in cashflows ],
          columns = ('payment date', 'start date', 'end date',
                     'rate', 'amount'),
          index=['']*len(cashflows))

,payment date,start date,end date,rate,amount
,"January 15th, 2015","October 15th, 2014","January 15th, 2015",0.20 %,0.051124
,"July 15th, 2015","January 15th, 2015","July 15th, 2015",0.20 %,0.100606
,"January 15th, 2016","July 15th, 2015","January 15th, 2016",0.20 %,0.102274
,"July 15th, 2016","January 15th, 2016","July 15th, 2016",0.20 %,0.101162
,"January 16th, 2017","July 15th, 2016","January 16th, 2017",0.20 %,0.102831
,"July 17th, 2017","January 16th, 2017","July 17th, 2017",0.20 %,0.101162
,"January 15th, 2018","July 17th, 2017","January 15th, 2018",0.20 %,0.101162
,"July 16th, 2018","January 15th, 2018","July 16th, 2018",0.20 %,0.101162
,"January 15th, 2019","July 16th, 2018","January 15th, 2019",0.20 %,0.101718
,"July 15th, 2019","January 15th, 2019","July 15th, 2019",0.20 %,0.100606


The first coupon has the correct dates, but the rate is wrong.  To use the right one, we have to build a custom first coupon with the correct index and use it instead of the current one.  We also need to set it a pricer (which is usually done for us by `IborLeg`).

In [25]:
first = as_floating_rate_coupon(cashflows[0])
coupon3m = IborCoupon(first.date(), first.nominal(),
                      first.accrualStartDate(), first.accrualEndDate(),
                      first.fixingDays(), index_3m)
coupon3m.setPricer(BlackIborCouponPricer())

cashflows = (coupon3m,) + cashflows[1:]

In [26]:
DataFrame([ (c.date(),
             as_coupon(c).accrualStartDate(), as_coupon(c).accrualEndDate(),
             utils.format_rate(as_coupon(c).rate()), c.amount())
            for c in cashflows ],
          columns = ('payment date', 'start date', 'end date',
                     'rate', 'amount'),
          index=['']*len(cashflows))

,payment date,start date,end date,rate,amount
,"January 15th, 2015","October 15th, 2014","January 15th, 2015",0.15 %,0.038341
,"July 15th, 2015","January 15th, 2015","July 15th, 2015",0.20 %,0.100606
,"January 15th, 2016","July 15th, 2015","January 15th, 2016",0.20 %,0.102274
,"July 15th, 2016","January 15th, 2016","July 15th, 2016",0.20 %,0.101162
,"January 16th, 2017","July 15th, 2016","January 16th, 2017",0.20 %,0.102831
,"July 17th, 2017","January 16th, 2017","July 17th, 2017",0.20 %,0.101162
,"January 15th, 2018","July 17th, 2017","January 15th, 2018",0.20 %,0.101162
,"July 16th, 2018","January 15th, 2018","July 16th, 2018",0.20 %,0.101162
,"January 15th, 2019","July 16th, 2018","January 15th, 2019",0.20 %,0.101718
,"July 15th, 2019","January 15th, 2019","July 15th, 2019",0.20 %,0.100606


As before, the resulting cash flows can be used to instantiate a bond or a swap.